# Modelo A para clasificación de imagenes

#### Jeffrey Daniel Leiva Cascante 2021016720
#### Richard Osvaldo León Chinchilla 2019003759

## Inicialización

In [6]:
import torch
print(torch.__version__)
print(torch.cuda.is_available())

from torchvision import transforms, datasets
from torch.utils.data import DataLoader, random_split

transform = transforms.Compose([
    transforms.Resize((224, 224)), # Resnet es de 224x224
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5], std=[0.5]),
    transforms.RandomRotation(20),  # Rotación aleatoria
    transforms.RandomHorizontalFlip(),  # Inversión horizontal aleatoria
    transforms.ColorJitter(brightness=0.2, contrast=0.2),  # Variación de brillo y contraste
])


train_dataset = datasets.ImageFolder(root='./Covid19-dataset-filters/train', transform=transform)

val_percent = 0.15

# Calculamos el tamaño de los conjuntos de entrenamiento y validación
val_size = int(val_percent * len(train_dataset))
train_size = len(train_dataset) - val_size

# Se divide el dataset en dos
train_dataset, val_dataset = random_split(train_dataset, [train_size, val_size])

# Creamos los dataloaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=True)

2.4.1+cu118
True


## Carga de Resnet-50 preentrenado

In [7]:
import torch.nn as nn
import torchvision.models as models

# Cargamos el modelo preentrenado
resnet50  = models.resnet50(pretrained=True)

# Se modifica la capa final para adaptarla a 3 clases
num_classes = 3
num_ftrs = resnet50.fc.in_features
resnet50.fc = nn.Linear(num_ftrs, num_classes)

# Se congela el modelo para que no se actualicen los pesos de las capas preentrenadas
for param in resnet50.parameters():
    param.requires_grad = False

for param in resnet50.fc.parameters():
    param.requires_grad = True

# Se mueve el modelo a la GPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
resnet50 = resnet50.to(device)

# Solo los parametros de la capa final se entrenan
optimizer = torch.optim.Adam(resnet50.fc.parameters(), lr=0.001)

# Se define la función de pérdida
criterion = nn.CrossEntropyLoss()

RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


## Ciclo de entrenamiento

In [3]:
# Epochs
n_epochs = 10

# Entrenamiento
for epoch in range(n_epochs):
    resnet50.train()
    train_loss = 0.0
    for input,labels in train_loader:
        inputs, labels = input.to(device), labels.to(device)
        optimizer.zero_grad() # Se ponen a cero los gradientes
        outputs = resnet50(inputs)
        loss = criterion(outputs, labels)

        loss.backward() # Se calculan los gradientes
        optimizer.step()

        train_loss += loss.item() * inputs.size(0)
    epoch_loss = train_loss / len(train_loader.dataset)
    print('Epoch {} / {} - Training Loss: {:.4f}'.format(epoch+1, n_epochs, epoch_loss))



RuntimeError: element 0 of tensors does not require grad and does not have a grad_fn

## Validación

In [ ]:
resnet50.eval()
correct = 0
total = 0
val_loss = 0.0

with torch.no_grad():
    for input, labels in val_loader:
        inputs, labels = input.to(device), labels.to(device)
        outputs = resnet50(inputs)
        loss = criterion(outputs, labels)

        val_loss += loss.item() * inputs.size(0)

        _, predicted = torch.max(outputs, 1)
        correct += (predicted == labels).sum().item()
        total += labels.size(0)

val_loss = val_loss / len(val_loader.dataset)
val_acc = correct / total
print('Validation Loss: {:.4f} - Validation Accuracy: {:.4f}'.format(val_loss, val_acc))